In [2]:
from DataClass.data_utils import create_vocab_dictionary

In [22]:
d, v = create_vocab_dictionary('..', 'all_tokens.pickle', False, 4500)

In [24]:
# ebmalen(d)

4997

In [15]:
# import pickle
# import pandas as pd
# import numpy as np

# x = np.loadtxt('Retrieve-and-Edit-Framework/datasets/word_vectors/github.txt', delimiter=' ')

In [19]:
d = pd.read_csv('Retrieve-and-Edit-Framework/datasets/word_vectors/github.txt', delimiter = ' ', header=None)

In [21]:
d[0]

0               .
1               )
2               (
3               §
4               ,
          ...    
3853       Lookup
3854    getPlugin
3855         sess
3856       Engine
3857          std
Name: 0, Length: 3858, dtype: object

In [1]:
from torch.utils.data import ConcatDataset, DataLoader
from transformer.bart import BartModel
from transformer.configuration_bart import BartConfig
from DataClass.torchData import *

model_config = BartConfig(
        vocab_size=len(word2idx), pad_token_id=0,
        eos_token_id=2, d_model=128,
        encoder_ffn_dim=512,
        encoder_layers=6,
        encoder_attention_heads=8,
        decoder_ffn_dim=512,
        decoder_layers=6,
        decoder_attention_heads=8,
        dropout=0.1,
        max_encoder_position_embeddings=386,
        max_decoder_position_embeddings=128)

model = BartModel(model_config)

In [12]:
model.decoder.layers.parameters()

<generator object Module.parameters at 0x145b8bde0>

In [2]:
checkpoint = torch.load("checkpoint-ruleseq2seq.pth", map_location=torch.device('cpu'))
hey = checkpoint['model']

a, b = zip(*hey.items())

for k, v in zip(a,b):
    hey[k[7:]] = v
    del hey[k]

In [3]:
model.load_state_dict(hey)

<All keys matched successfully>

In [27]:
line = "def send_css(path):"
actual_y = "return send_from_directory('static/css', path)"
retx = "def send_js(path):"
rety = "return  send_from_directory('static/js', path)"

In [28]:
from DataClass.data_utils import preprocess_tokens, tokenize_fine_grained
from DataClass.torchData import word2idx, UNKNOWN_IDX, idx2word
SEP_CONTEXT_WORD = '<SEP_CONTEXT>'
SEP_PAIR_WORD = '<SEP_PAIR>'
SEP_RET_WORD = '<SEP_RET>'

# line = 'if tool_name == "mllr":'#"domain = config.get('instance', 'domain')"
context_tokens = preprocess_tokens(tokenize_fine_grained(line), 128) + [SEP_CONTEXT_WORD]
# retx = 'elif tool_name == "vtln":'#'domain = config.get("instance", "domain")'
retx_tokens = preprocess_tokens(tokenize_fine_grained(retx), 128)
# rety = 'command.extend(["-v", "vtln"])'#'headers  = {"Authorization" : "Bearer %s" % token}'
rety_tokens = preprocess_tokens(tokenize_fine_grained(rety), 128)
context_tokens += retx_tokens
context_tokens += [SEP_RET_WORD]
context_tokens += rety_tokens
in_ = torch.LongTensor([word2idx.get(token, UNKNOWN_IDX) for token in context_tokens]).contiguous().view(1, 386)

dec = ""
y_tokens = preprocess_tokens(tokenize_fine_grained(dec), 128)
y_tokens = y_tokens[:y_tokens.index("<EOS>")]
deco = [word2idx.get(token, UNKNOWN_IDX) for token in y_tokens]

In [29]:
# deco = y_tokens
dec_ = torch.LongTensor(deco).contiguous().view(1, len(deco))
for i in range(127 - len(y_tokens)):
    hey = model(input_ids = in_, decoder_input_ids=dec_)[0, -1, :]
    new_char = int(hey.max(0)[1])
    if new_char == UNKNOWN_IDX:
        new_char = context_tokens[i+1]
    deco += [new_char]
    if int(hey.max(0)[1]) == 2: break
#     y_tokens = preprocess_tokens(tokenize_fine_grained(deco), 128)[:i]
    dec_ = torch.LongTensor(deco).contiguous().view(1, len(deco))
    
''.join([idx2word[idx] for idx in deco])

"<BOS>returnsend_from_directory('static/js',path)<EOS>"

In [30]:
# actual_y = "headers = {'Authorization': 'Bearer %s' % token}"
''.join(preprocess_tokens(tokenize_fine_grained(actual_y), 128))

"<BOS>returnsend_from_directory('static/css',path)<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>"

In [1]:
from torch.utils.data import ConcatDataset, DataLoader
from transformer.bart import BartModel
from transformer.configuration_bart import BartConfig
from DataClass.torchData import *

model_config = BartConfig(
        vocab_size=len(word2idx), pad_token_id=0,
        eos_token_id=2, d_model=128,
        encoder_ffn_dim=512,
        encoder_layers=6,
        encoder_attention_heads=8,
        decoder_ffn_dim=512,
        decoder_layers=6,
        decoder_attention_heads=8,
        dropout=0.1,
        max_encoder_position_embeddings=128,
        max_decoder_position_embeddings=128)

model = BartModel(model_config)

In [2]:
model.shared.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0221, -0.0118, -0.0161,  ..., -0.0460,  0.0221,  0.0192],
        [-0.0207, -0.0227,  0.0015,  ...,  0.0046,  0.0218,  0.0117],
        ...,
        [-0.0132,  0.0028, -0.0170,  ...,  0.0057, -0.0064, -0.0052],
        [ 0.0198,  0.0163, -0.0217,  ..., -0.0164,  0.0257,  0.0467],
        [ 0.0345,  0.0384, -0.0264,  ..., -0.0031, -0.0233,  0.0296]],
       requires_grad=True)

In [3]:
checkpoint = torch.load("checkpoint-120000.pth", map_location=torch.device('cpu'))
hey = checkpoint['model']

a, b = zip(*hey.items())

for k, v in zip(a,b):
    hey[k[7:]] = v
    del hey[k]

In [4]:
model.load_state_dict(hey)

<All keys matched successfully>

In [5]:
model.shared.weight

Parameter containing:
tensor([[ 0.0046, -0.0076, -0.0695,  ..., -0.0314, -0.0249, -0.0206],
        [-0.0230, -0.0604, -0.1208,  ..., -0.0293, -0.0233, -0.0037],
        [ 0.0137, -0.0524, -0.0123,  ...,  0.0865, -0.0476, -0.1183],
        ...,
        [ 0.2625, -0.0448,  0.2158,  ...,  0.0953,  0.1499,  0.0506],
        [ 0.1097, -0.0928,  0.0395,  ..., -0.0195,  0.1533, -0.2183],
        [ 0.0278, -0.1492, -0.1252,  ...,  0.1920, -0.1664, -0.1495]],
       requires_grad=True)

In [6]:
from DataClass.data_utils import preprocess_tokens, tokenize_fine_grained
from DataClass.torchData import word2idx, UNKNOWN_IDX, idx2word

line = "if x == 5: break"
context_tokens = preprocess_tokens(tokenize_fine_grained(line), 128)
in_ = torch.LongTensor([word2idx.get(token, UNKNOWN_IDX) for token in context_tokens]).contiguous().view(1, 128)

dec = ""
y_tokens = preprocess_tokens(tokenize_fine_grained(dec), 128)
y_tokens = y_tokens[:y_tokens.index("<EOS>")]
deco = [word2idx.get(token, UNKNOWN_IDX) for token in y_tokens]

In [10]:
context_tokens

['<BOS>',
 'if',
 'x',
 '=',
 '=',
 '5',
 ':',
 'break',
 '<EOS>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<

In [7]:
dec_ = torch.LongTensor(deco).contiguous().view(1, len(deco))

In [8]:
# deco = y_tokens
for i in range(127 - len(y_tokens)):
    hey = model(input_ids = in_, decoder_input_ids=dec_)[0, -1, :]
    deco += [int(hey.max(0)[1])]
    if int(hey.max(0)[1]) == 2: break
#     y_tokens = preprocess_tokens(tokenize_fine_grained(deco), 128)[:i]
    dec_ = torch.LongTensor(deco).contiguous().view(1, len(deco))

In [9]:
''.join([idx2word[idx] for idx in deco])

'<BOS>ifx==5:continue<EOS>'

In [182]:
''.join(translate)

'_self._done=False<EOS>'

In [21]:
import torch.optim as optim

In [24]:
optim.Adam(x[0], lr=1e-2)

TypeError: params argument given to the optimizer should be an iterable of Tensors or dicts, but got torch.FloatTensor

In [23]:
optim.Adam(d, lr=1e-2)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)

In [1]:
x = (1, 2)
b = (4,)
new = b+x

In [2]:
new

(4, 1, 2)

In [9]:
from DataClass.MetaTorchData import MetaRepo, MetaRetrieved
from torch.utils.data import DataLoader
ds = MetaRepo('repo_files/beaker_line_pairs.csv', False, k_shot=3)
d = DataLoader(ds, shuffle=True)
loader = iter(d)

In [10]:
ax, bx, cx, dx = next(loader)

In [11]:
ax.shape

torch.Size([1, 3, 128])

In [1]:
from transformer.Models import Transformer

In [58]:
model = Transformer(100, 100, 0, 0)

temp_params = list(model.parameters())

src_emb_params = list(model.encoder.src_word_emb.parameters())
trg_emb_params = list(model.decoder.trg_word_emb.parameters())

In [60]:
temp_params.remove(src_emb_params[0])
# temp_params.remove(trg_emb_params)
# trg_emb_params[0]

RuntimeError: The size of tensor a (512) must match the size of tensor b (100) at non-singleton dimension 0

In [46]:
# list(d.encoder.src_word_emb.parameters())[0][0]
hey = list(d.parameters())

In [48]:
hey.remove([list(d.encoder.src_word_emb.parameters())[0]), list(d.decoder.trg_word_emb.parameters())[0]])

TypeError: eq() received an invalid combination of arguments - got (list), but expected one of:
 * (Tensor other)
      didn't match because some of the arguments have invalid types: ([31;1mlist[0m)
 * (Number other)
      didn't match because some of the arguments have invalid types: ([31;1mlist[0m)


In [47]:
# len(list(d.parameters()))
len(hey)

185

In [14]:
len(list(d.parameters()) - list(d.encoder.src_word_emb.parameters()))

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [10]:
import pandas as pd
import csv
from DataClass.Constants import NO_CONTEXT_WORD, UNKNOWN_IDX
import numpy as np

In [11]:
filename = 'repo_files/lantern_line_pairs.csv'

In [23]:
from DataClass.data_utils import fix_quote_strings, tokenize_fine_grained, preprocess_tokens
idx = 100
try:
    x = next(pd.read_csv(filename,
                                skiprows=idx * 1+1,
                                chunksize=1, header=None, dtype=str)).fillna(NO_CONTEXT_WORD).values
except:
        x = next(pd.read_csv(filename,
                            skiprows=idx * 1+1,
                            chunksize=1, header=None,
                            sep=',\s+', quoting=csv.QUOTE_ALL, dtype=str)).fillna(NO_CONTEXT_WORD).values
        x = np.array(fix_quote_strings(x[0, 0]))
        

In [24]:
x

array([['        self._p_transmit = self._p.getCharacteristics(uuid=self.TRANSMIT_UUID)[0]',
        '    def disconnect(self):']], dtype=object)

In [27]:
from DataClass.data_utils import create_vocab_dictionary

word2idx, idx2ord = create_vocab_dictionary(path='..', file='all_tokens.pickle')

In [33]:
import pickle
all_tokens = pickle.load(open('../all_tokens.pickle', 'rb'))

In [35]:
all_tokens['self']

5

In [38]:
word2idx['self']

KeyError: 'self'

In [36]:
d = tokenize_fine_grained(x[0, 0])

['self',
 '.',
 '_',
 'p',
 '_',
 'transmit',
 '=',
 'self',
 '.',
 '_',
 'p',
 '.',
 'get',
 'Characteristics',
 '(',
 'uuid',
 '=',
 'self',
 '.',
 'TRANSMIT',
 '_',
 'UUID',
 ')',
 '[',
 '0',
 ']']

In [30]:
[word2idx.get(token, UNKNOWN_IDX) for token in d]

[4,
 4,
 4,
 4,
 4,
 344,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1280,
 4,
 4,
 4,
 4]

In [31]:
d

['self',
 '.',
 '_',
 'p',
 '_',
 'transmit',
 '=',
 'self',
 '.',
 '_',
 'p',
 '.',
 'get',
 'Characteristics',
 '(',
 'uuid',
 '=',
 'self',
 '.',
 'TRANSMIT',
 '_',
 'UUID',
 ')',
 '[',
 '0',
 ']']

In [1]:
from DataClass.MetaTorchData import MetaRepo, MetaRetrieved
from torch.utils.data import DataLoader
ds = MetaRepo('repo_files/beaker_line_pairs.csv', False)
d = DataLoader(ds, shuffle=True)
loader = iter(d)
ax, bx, cx, dx = next(loader)

In [2]:
import pandas as pd
df = pd.read_csv('./repo_files/beaker_line_pairs.csv')

In [3]:
df2 = pd.concat([df, df, df, df], axis=1)

In [4]:
df2.to_csv('lets_check.csv', index=None)

In [3]:
ds = MetaRepo('repo_files/beaker_line_pairs.csv', False)
d = DataLoader(ds, shuffle=True)

In [4]:
loader = iter(d)
ax, bx, cx, dx = next(loader)

In [5]:
ax.shape

torch.Size([1, 5, 128])

In [5]:
# ds = MetaRepo('./repo_files/beaker_line_pairs.csv', False)
ds = MetaRepo('lets_check.csv', True, 3)
d = DataLoader(ds, shuffle=True, batch_size=1)

In [6]:
loader = iter(d)

In [7]:
support_x, support_y, query_x, query_y = next(loader)

In [8]:
support_x

tensor([[[  1, 505,  21,  ...,   0,   0,   0],
         [  1,   4,  19,  ...,   0,   0,   0],
         [  1,  13,   4,  ...,   0,   0,   0],
         [  1, 220, 619,  ...,   0,   0,   0],
         [  1,  36, 147,  ...,   0,   0,   0]]])

In [40]:
support_y.shape

torch.Size([1, 5, 128])

In [41]:
query_x.shape

torch.Size([1, 1, 902])

In [42]:
query_y.shape

torch.Size([1, 1, 128])

In [24]:
h = y[(y['line'].str.lstrip().str.rstrip().str.len() > 0) & (~y['line'].str.contains('import'))].reset_index(drop=True)
h

,line
0,ahe sf sdlkgh ag s
1,Idk flah fgq e
2,idk if you should be doing this anyways ok


0     0
1    19
2    18
3    42
4    10
Name: line, dtype: int64

In [22]:
import pandas as pd

y = pd.DataFrame([['    '], [' ahe sf  sdlkgh ag s'], ['Idk  flah fgq e   '], ['idk if you should be doing this anyways ok'], ['import sys']])
y.columns = ['line']

In [15]:
h.values[0]

array([' ahe sf  sdlkgh ag s'], dtype=object)

In [1]:
import pickle
import numpy as np
import pandas as pd
import os, csv
from torch.utils.data import ConcatDataset, DataLoader
from DataClass.torchData import PairDataset, batch_collate_fn
import torch

In [2]:
datasets = list(filter(lambda x: True if x.endswith('.csv') else False, next(os.walk('./repo_files'))[2]))
data_loader = DataLoader(ConcatDataset([PairDataset('./repo_files' + '/' + dataset) for dataset in datasets]), 
                        batch_size=16, 
                        shuffle=True, 
                        collate_fn=batch_collate_fn)

In [3]:
from tqdm import tqdm

for x, y in tqdm(data_loader):
    batch_x = x
    batch_y = y

  0%|          | 872/2095149 [05:04<170:10:21,  3.42it/s]

KeyboardInterrupt: 

In [15]:
filename = './repo_files/flaskjinja_line_pairs.csv'
x = next(pd.read_csv(filename,
                                skiprows=3456 * 1+1,
                                chunksize=1, header=None, dtype=str)).fillna('OSOFo').values

In [19]:
len(x[0])

2

In [8]:
batch_x[0]

tensor([    1,   510,    10,    13,   173,    53,   256,    53, 30466,    53,
         1363,    16,    18,    11,    11, 34486,    53,  1363,     2,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [69]:
def fix(v):
    idx = v.find('","')
    x, y = v[1:idx], v[idx+3:-1]
    if "\'" in x:
        x = x.replace('"', "'").replace("\t", ' ')
    else:
        x = x.replace('""', '"').replace("'", '"').replace('\t', ' ')

    if "\'" in y:
        y = y.replace('"', "'").replace("\t", ' ')
    else:
        y = y.replace('""', '"').replace("'", '"').replace('\t', ' ')
    
    return [[x, y]]

In [13]:
h = next(pd.read_csv('./repo_files/Milis-Yukleyici_line_pairs.csv', skiprows=1+32, chunksize=1, header=None))

In [2]:
def batch_collate_fn(data):
        data = list(filter(lambda z : z is not None, data))
        x, y = zip(*data)
        return x, y
#         x = pd.DataFrame(x)
#         y = pd.DataFrame(y)
        
#         x = np.where(x.isin(word2idx.keys()), x.replace(word2idx), UNKNOWN_IDX).astype('int64')
#         y = np.where(y.isin(word2idx.keys()), y.replace(word2idx), UNKNOWN_IDX).astype('int64')

#         batch_xs = torch.LongTensor(x.values)
#         batch_ys = torch.LongTensor(y.values)
        return batch_xs, batch_ys

In [4]:
d = pickle.load(open('pickle_files/10DaysOfStatistics_line_pairsdataset_edit.pkl', 'rb'))

In [79]:
lantern = pd.read_csv('./repo_files/lantern_line_pairs.csv')
electron = pd.read_csv('./repo_files/electron_line_pairs.csv')

In [80]:
electron = pd.concat([electron, electron, electron], axis=1)
lantern = pd.concat([lantern, lantern, lantern], axis=1)

In [101]:
import pandas as pd
import torch, os, pickle
from torch.utils.data import Dataset, Dataset, ConcatDataset, DataLoader
from DataClass.data_utils import tokenize_fine_grained, create_vocab_dictionary
import numpy as np
from DataClass.Constants import NO_CONTEXT_WORD, UNKNOWN_WORD, PAD_WORD

tokens_file = './repo_files/all_tokens.pickle'
tokens_dict = pickle.load(open(tokens_file, 'rb'))
tokens_dict, _ = create_vocab_dictionary(tokens_dict)
UNKNOWN_IDX = tokens_dict[UNKNOWN_WORD]
MAX_DIMENSION = 10

def preprocess_tokens(tokens, max_dim):
    tokens = [START_WORD] + tokens
    n = len(tokens) + 1
    # minus one since end word needs to go on too
    tokens = tokens[:min(n, max_dim-1)] + [END_WORD] + [PAD_WORD]*max(0, max_dim-n)
    return tokens

def preprocess_context(context, n, max_dim):
    context_tokens = preprocess_tokens(tokenize_fine_grained(context[0, 0]), max_dim)
    context_tokens += [SEP_CONTEXT_WORD]
    
    for idx in range(n-1):
        context_tokens += preprocess_tokens(tokenize_fine_grained(context[0, idx*2-1]), max_dim) + [SEP_PAIR_WORD]
        context_tokens += preprocess_tokens(tokenize_fine_grained(context[0, idx*2]), max_dim) + [SEP_RET_WORD]
    
    context_tokens += preprocess_tokens(tokenize_fine_grained(context[0, -2]), max_dim) + [SEP_PAIR_WORD]
    context_tokens += preprocess_tokens(tokenize_fine_grained(context[0, -1]), max_dim)
    return context_tokens


class MetaRetrieveDataset(Dataset):

    def __init__(self, filename, chunksize, n_retrieved, k_shot, meta_search_range=100):
        
        self.filename = filename
        self.chunksize = 1 # more than this and requires a lot more changes in collate fn
        temp = next(pd.read_csv(self.filename, skiprows = 0, chunksize=1, header=None))
        self.n_retrieved = n_retrieved
        self.max_dim = MAX_DIMENSION
        self.meta_search_range = meta_search_range
        self.k_shot = k_shot
        self.len = int(temp.values[0][0] / self.chunksize)
        
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        x = next(pd.read_csv(self.filename,
                            skiprows=idx * self.chunksize+1,
                            chunksize=self.chunksize, header=None)).fillna(NO_CONTEXT_WORD).values

        support_sample_idxs = np.random.randint(0, self.len, (self.k_shot))
        while(np.where(np.logical_end(support_sample_idxs >= idx-meta_search_range, support_sample_idxs <= idx+meta_search_range))):
            support_sample_idxs = np.random.randint(0, self.len, (self.k_shot))
        
        support_context_tokens = []
        support_y_tokens = []
        for support_idx in support_sample_idxs:
            support_temp = next(pd.read_csv(self.filename,
                            skiprows=support_idx * self.chunksize+1,
                            chunksize=self.chunksize, header=None)).fillna(NO_CONTEXT_WORD).values
            
            support_context_tokens.append(preprocess_context(support_temp[:, :-1], self.n_retrieved, self.max_dim))
            support_y_tokens.append(preprocess_tokens(tokenize_fine_grained(support_temp[0, -1]), self.max_dim))
        
        context_tokens = preprocess_context(x[:, :-1], self.n_retrieved, self.max_dim)
        y_tokens = preprocess_tokens(tokenize_fine_grained(x[0, -1]), self.max_dim)
        return (support_context_tokens, support_y_tokens, context_tokens, y_tokens)
    
def batch_collate_fn(data):
#         print(data)
        x, y = zip(*data)
        x = pd.DataFrame(support_c)
#         x = pd.DataFrame(x)
#         y = pd.DataFrame(y)
        x = np.where(x.isin(tokens_dict.keys()), x.replace(tokens_dict), UNKNOWN_IDX).astype('int64')
        y = np.where(y.isin(tokens_dict.keys()), y.replace(tokens_dict), UNKNOWN_IDX).astype('int64')

        batch_xs = torch.LongTensor(x)
        batch_ys = torch.LongTensor(y)
        return batch_xs, batch_ys
    
    


# def createDataLoaderAllFiles(dataset_dir, dataset_class=PairDatasetLazy, shuffle=True, batch_size=128):
#     datasets = list(filter(lambda x: True if x.endswith('.csv') else False, next(os.walk(dataset_dir))[2]))
#     datasets = list(filter(lambda x: True if x.startswith('check') else False, next(os.walk(dataset_dir))[2]))
#     return DataLoader(ConcatDataset([dataset_class(dataset_dir + '/' + dataset, chunksize=batch_size, n_retrieved=RETRIEVED_EXAMPLES) for dataset in datasets]), 
#                         batch_size=batch_size, 
#                         shuffle=shuffle, 
#                         collate_fn=batch_collate_fn)






In [104]:
# d = createDataLoaderAllFiles('./repo_files', RetrieveDataset, batch_size=2)

In [105]:
for x, y in d:
    break

(['<BOS>', 'return', 'os', '.', 'path', '.', 'basename', '(', 'binary', '<EOS>', '<SEP_CONTEXT>', '<BOS>', 'return', 'os', '.', 'path', '.', 'basename', '(', 'binary', '<EOS>', '<SEP_PAIR>', '<BOS>', 'return', 'os', '.', 'path', '.', 'basename', '(', 'binary', '<EOS>', '<SEP_RET>', '<BOS>', 'def', 'main', '(', ')', ':', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<SEP_PAIR>', '<BOS>', 'return', 'os', '.', 'path', '.', 'basename', '(', 'binary', '<EOS>'], ['<BOS>', 'upload', '_', 'options', '.', 'server', '=', 'server', '<EOS>', '<PAD>', '<SEP_CONTEXT>', '<BOS>', 'upload', '_', 'options', '.', 'server', '=', 'server', '<EOS>', '<PAD>', '<SEP_PAIR>', '<BOS>', 'upload', '_', 'options', '.', 'server', '=', 'server', '<EOS>', '<PAD>', '<SEP_RET>', '<BOS>', 'upload', '_', 'options', '.', 'save', '_', 'cookies', '=', '<EOS>', '<SEP_PAIR>', '<BOS>', 'upload', '_', 'options', '.', 'server', '=', 'server', '<EOS>', '<PAD>'])


NameError: name 'support_c' is not defined

In [79]:
y.shape

torch.Size([12, 128])

In [58]:
hey = next(pd.read_csv('./repo_files/check_lantern_retrieve.csv', skiprows=1, chunksize=1, header=None)).fillna(NO_CONTEXT_WORD).values[:, :-1]

In [61]:
tokenize_fine_grained(hey[0, 2])

['OSOFo']

In [99]:
lantern.columns = [lantern.columns[0]] + [0]*(len(lantern.columns)-1)

In [83]:
electron.columns = [electron.columns[0]] + [0]*(len(electron.columns)-1)

In [100]:
lantern.to_csv('./repo_files/check_lantern_retrieve.csv', index=None)

In [65]:
idx = 0

x = next(pd.read_csv('./repo_files/check_lantern_retrieve.csv',
                            skiprows=idx * 1+1,
                            chunksize=1, header=None)).fillna(NO_CONTEXT_WORD).values

In [66]:
x

array([['OSOFo',
        'from mercurial import cmdutil, commands, hg, util, error, match',
        'OSOFo',
        'from mercurial import cmdutil, commands, hg, util, error, match',
        'OSOFo',
        'from mercurial import cmdutil, commands, hg, util, error, match']],
      dtype=object)

In [94]:
pd.read_csv('./repo_files/check_electron_retrieve.csv')

,2579,0,0.1,0.2,0.3,0.4
0,NaN,import os,NaN,import os,NaN,import os
1,import os,import subprocess,import os,import subprocess,import os,import subprocess
2,import subprocess,import sys,import subprocess,import sys,import subprocess,import sys
3,import sys,TEMPLATE =,import sys,TEMPLATE =,import sys,TEMPLATE =
4,TEMPLATE =,def main():,TEMPLATE =,def main():,TEMPLATE =,def main():
...,...,...,...,...,...,...
2574,for dirname in args:,try:,for dirname in args:,try:,for dirname in args:,try:
2575,try:,os.makedirs(dirname),try:,os.makedirs(dirname),try:,os.makedirs(dirname)
2576,os.makedirs(dirname),except OSError as e:,os.makedirs(dirname),except OSError as e:,os.makedirs(dirname),except OSError as e:
2577,except OSError as e:,if e.errno == os.errno.EEXIST:,except OSError as e:,if e.errno == os.errno.EEXIST:,except OSError as e:,if e.errno == os.errno.EEXIST:


In [27]:
import pandas as pd
import torch, os, pickle
from torch.utils.data import Dataset, Dataset, ConcatDataset, DataLoader
from DataClass.data_utils import tokenize_fine_grained, create_vocab_dictionary
import numpy as np
from DataClass.Constants import PAD_WORD, START_WORD, END_WORD, PAD_IDX, START_IDX, END_IDX, NO_CONTEXT_IDX, NO_CONTEXT_WORD, UNKNOWN_IDX, UNKNOWN_WORD

tokens_file = './repo_files/all_tokens.pickle'
tokens_dict = pickle.load(open(tokens_file, 'rb'))
tokens_dict, _ = create_vocab_dictionary(tokens_dict)
UNKNOWN_IDX = tokens_dict[UNKNOWN_WORD]
MAX_DIMENSION = 128

class PairDatasetLazy(Dataset):

    def __init__(self, filename, chunksize):
        
        self.filename = filename
        self.chunksize = 1 # more than this and requires a lot more changes in collate fn
        temp = next(pd.read_csv(self.filename, skiprows = 0, chunksize=1, header=None))
        self.max_dim = MAX_DIMENSION-2
        self.len = int(temp.values[0][0] / self.chunksize)
        
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        x = next(pd.read_csv(self.filename,
                            skiprows=idx * self.chunksize+1,
                            chunksize=self.chunksize, header=None, dtype=str)).fillna(NO_CONTEXT_WORD).values

        x_tokens = tokenize_fine_grained(x[0, 0])
        y_tokens = tokenize_fine_grained(x[0, 1])

        x_n = len(x_tokens); y_n = len(y_tokens)

        x_tokens = [START_WORD] + x_tokens[:(self.max_dim) if self.max_dim < x_n else x_n] + [PAD_WORD]*(max(0, self.max_dim-x_n)) + [END_WORD]
        y_tokens = [START_WORD] + y_tokens[:(self.max_dim) if self.max_dim < y_n else y_n] + [PAD_WORD]*(max(0, self.max_dim-y_n)) + [END_WORD]

        return x_tokens, y_tokens
        # return np.array(x_tokens).reshape(-1, 1), np.array(y_tokens).reshape(-1, 1)
    
def batch_collate_fn(data):
        x, y = zip(*data)
        
        x = pd.DataFrame(x)#.fillna(PAD_WORD)
        y = pd.DataFrame(y)#.fillna(PAD_WORD)
        x = np.where(x.isin(tokens_dict.keys()), x.replace(tokens_dict), UNKNOWN_IDX).astype('int64')
        y = np.where(y.isin(tokens_dict.keys()), y.replace(tokens_dict), UNKNOWN_IDX).astype('int64')

        batch_xs = torch.LongTensor(x)
        batch_ys = torch.LongTensor(y)
        # x = pd.DataFrame(x).replace(tokens_dict).fillna(PAD_IDX)
        # y = pd.DataFrame(y).replace(tokens_dict).fillna(PAD_IDX)

        # batch_xs = torch.LongTensor(x.values)
        # batch_ys = torch.LongTensor(y.values)
        return batch_xs, batch_ys
    
    

def createDataLoaderAllFiles(dataset_dir, dataset_class=PairDatasetLazy, shuffle=True, batch_size=128):
    datasets = list(filter(lambda x: True if x.endswith('.csv') else False, next(os.walk(dataset_dir))[2]))
    return DataLoader(ConcatDataset([dataset_class(dataset_dir + '/' + dataset, chunksize=batch_size) for dataset in datasets]), 
                        batch_size=batch_size, 
                        shuffle=shuffle, 
                        collate_fn=batch_collate_fn)






In [1]:
import os
import re
import glob
import time
import pickle
import random
import distance
import collections
import configargparse
from tqdm import tqdm
from  more_itertools import unique_everseen

# import opts

from flask import Flask, render_template, url_for, redirect, jsonify, request  # noqa
from flask_cors import CORS

In [1]:
from DataClass.Crawler import Crawler
c = Crawler()
c.tokenize_from_files('./repo_files')

On repo [1/4]
On repo [2/4]
On repo [3/4]
On repo [4/4]


In [1]:
from DataClass.torchData import createDataLoaderAllFiles

In [2]:
z = createDataLoaderAllFiles('./repo_files')
# for i, (x, y) in 

In [3]:
for i, (x, y) in enumerate(z):
    break

[(['<BOS>', 'return', 'retcode', '<EOS>'], ['<BOS>', 'if', '_', '_', 'name', '_', '_', '=', '=', "'", '_', '_', 'main', '_', '_', "'", ':', '<EOS>']), (['<BOS>', 'return', 'len', '(', 'broken', 'Links', ')', '<EOS>'], ['<BOS>', 'def', 'check', 'Sections', '(', 'sections', ',', 'lines', ')', ':', '<EOS>']), (['<BOS>', 'raise', 'hg', '_', 'util', '.', 'Abort', '(', 'codereview', '_', 'disabled', ')', '<EOS>'], ['<BOS>', 'dirty', '=', '{', '}', '<EOS>']), (['<BOS>', 'root', '=', "'", "'", '<EOS>'], ['<BOS>', 'try', ':', '<EOS>']), (['<BOS>', "'", 'libffmpeg', '.', 'so', "'", ',', '<EOS>'], ['<BOS>', "'", 'lib', 'GLESv', '2', '.', 'so', "'", ',', '<EOS>']), (['<BOS>', 'line', '=', 'line', '.', 'strip', '(', ')', '<EOS>'], ['<BOS>', 'if', 'line', '=', '=', "'", "'", ':', '<EOS>']), (['<BOS>', 'return', 'newpatch', ',', '"', '"', '<EOS>'], ['<BOS>', 'if', 'not', 'm', ':', '<EOS>']), (['<BOS>', "'", 'You', 'have', 'to', 'pass', '-', '-', 'overwrite', 'to', 'overwrite', 'a', 'published', 'rele

In [4]:
x

tensor([[   1,  110,   12,  ...,    0,    0,    0],
        [   1,   42,    6,  ...,    0,    0,    0],
        [   1,  729,  266,  ...,    0,    0,    0],
        ...,
        [   1, 1152,    9,  ...,    0,    0,    0],
        [   1,  100,   22,  ...,    0,    0,    0],
        [   1,  146,    9,  ...,    0,    0,    0]])

In [6]:
y[0]

tensor([   1, 1028,   20,   57,   20,  117,   15,   72,   16,    5,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])

In [2]:
from EditorNoRet import EditorNoRetrieval

SyntaxError: invalid syntax (torchData.py, line 12)

In [3]:

hey = EditorNoRetrieval(num_layers=1, num_heads=1)

NameError: name 'EditorNoRetrieval' is not defined

In [4]:
import torch.optim as optim
optimizer = optim.Adam(hey.model.parameters(), lr=1e-2)
hey.train(optimizer)

NameError: name 'hey' is not defined

In [5]:
for i, (x, y) in enumerate(hey.data_loader):
    break

NameError: name 'hey' is not defined

In [6]:
# Basic Flask setup
# app = Flask(__name__)
# CORS(app)

# app.config['SECRET_KEY'] = 'fe093b0354f9ba0b1237a5e36f58caf55cc9f5682c2627b7463c30f0bbd97672'  # noqa


# Regex
newline_ptr = re.compile(r'(?:"[^"]*"|.)+')  # \n outside of quotes (https://stackoverflow.com/questions/24018577/parsing-a-string-in-python-how-to-split-newlines-while-ignoring-newline-inside)
comment_ptr = re.compile(r'"""(.*?)"""|\'\'\'(.*?)\'\'\'|#[^"\']*?(?=\n)|#.*?(".*?"|\'.*?\').*?(?=\n)', re.DOTALL|re.MULTILINE)
literal_ptr = re.compile(r'".*?"|\'.*?\'|[-+]?\d*\.\d+|\d+')
camelcase_ptr = re.compile(r"(?<=[a-z])([A-Z]+[a-z]*)")
number_ptr = re.compile(r'(?<=[^a-zA-Z])([-+]?\d*\.\d+|\d+)')
number_with_alpha_ptr = re.compile(r'(?<=[a-zA-Z])([-+]?\d*\.\d+|\d+)')  # split numbers from alpha
string_ptr = re.compile(r'".*?"|\'.*?\'')
code_ptr = re.compile(r"([^a-zA-Z0-9])")
whitespace_ptr = re.compile(r"(\s+)")


# Read default options
# parser = configargparse.ArgumentParser(description="index.py")
# opts.system_opts(parser)
# opt = parser.parse_args()

results = None

In [7]:
from DataClass.torchData import createDataLoaderAllFiles

SyntaxError: invalid syntax (torchData.py, line 12)

In [8]:
# path = "scrape_github/electron"


dl = 

SyntaxError: invalid syntax (<ipython-input-8-9db71dc3d418>, line 4)

In [9]:
def split_newlines(s):
    """
    Split based on new lines (\n) outside of quotes.

    Note that this coalesces several newlines into one,
    as blank lines are ignored. To avoid that, give a null case:

    (?:"[^"]*"|.)+|(?!\Z)
    """
    return re.findall(newline_ptr, s)

def remove_comments(source):
    return re.sub(r"\n\n+", "\n\n", re.sub(comment_ptr, "", source))


def get_lines_from_source(source):
    """
    Remove comments and empty lines from source.
    Return a list of lines
    """
    source = remove_comments(source)

    lines = split_newlines(source)
    return lines

In [10]:
def extract_examples_line_by_line(source):  # If line, no need to filter based on length
    examples = []

    lines = get_lines_from_source(source)
    if not lines:
        return []
    
    return lines

In [11]:
pickle.dump(token_dict, open('token_dict.pickle', 'wb'))

NameError: name 'token_dict' is not defined

In [12]:
from DataClass.data_utils import tokenize_fine_grained
import pandas as pd

In [16]:
def preprocess_source(source):
    """
    Remove # in strings for comment_ptr.

    TODO. Later on, fix regex instead.
    """
    for string in re.findall(r'".*?#.*?"|\'.*?#.*?\'', source):
        if '#' in string:
            modified = string.replace('#', '')
            source = source.replace(string, modified)
    return source


def read_data(path):
    if not os.path.exists(path):
        print("[!] Data does not exist")
    elif os.path.isfile(path):
        return read_file(path)
    else:
        return read_dir(path)


def read_file(path):
    num_file = 1
    sources = []
    try:
        with open(path, 'r') as f:
            source.append(preprocess_source(f.read()))
    except Exception as e:
        print(e)
    return sources, num_file


def read_dir(path):
    num_file = 0
    sources = []
    for filename in glob.iglob(os.path.join(path, "**/*.py"), recursive=True):
        if not filename.endswith('.py'): continue
        num_file += 1
        with open(filename, "r") as f:
            sources.append(preprocess_source(f.read()))
    return sources, num_file


token_dict = {}
import numpy as np

path = 'scrape_github/lantern'
sources, num_file = read_data(path)
all_lines = None
for source in sources:
    lines = extract_examples_line_by_line(source)
    if lines == []: continue
    for line in lines:
        for token in tokenize_fine_grained(line):
            token_dict[token] = token_dict.get(token, len(token_dict))
    y = pd.DataFrame(lines)
    y.columns = ['line']
    y = y[y['line'].apply(lambda x: len(str(x).strip()) > 0)].reset_index(drop=True)    
    x = pd.concat([pd.DataFrame([""]), y['line'][:-1]]).reset_index(drop=True)
    pair = pd.concat([x, y], axis=1)
    all_lines = pd.concat([all_lines, pair], axis=0)# if all_lines is not None else pair

all_lines = pd.concat([pd.DataFrame(np.array([all_lines.shape[0], None]).reshape(1, -1), columns=all_lines.columns), all_lines], axis=0)
all_lines.to_csv(path[len(path) - path[::-1].find('/'):] + '_line_pairs.csv', header=None, index=None)
    
    

In [15]:
import numpy as np
from torch.utils.data import Dataset, Dataset, ConcatDataset, DataLoader
import pandas as pd
import torch

class MyDataset(Dataset):

    def __init__(self, filename, chunksize):
        
        self.filename = filename
        self.chunksize = 2#chunksize
        temp = next(pd.read_csv(self.filename, skiprows = 0, chunksize=1, header=None))
        self.len = int(temp.values[0][0] / self.chunksize)
        
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        x = next(pd.read_csv(self.filename,
                            skiprows=idx * self.chunksize+1,
                            chunksize=self.chunksize, header=None, dtype=str)).fillna('OSOFo').values
        x_tokens = ['BOS'] + tokenize_fine_grained(x[0, 0]) + ['EOS']# for idx in range(self.chunksize)]
        y_tokens = ['BOS'] + tokenize_fine_grained(x[0, 1]) + ['EOS']# for idx in range(self.chunksize)]

        return x_tokens, y_tokens
    
    
def batch_collate_fn(data):
        x, y = zip(*data)#pd.DataFrame(zip(*h)).T
        return x, y
#         print(x)
        # off by one in tokenize dictionary
        x = pd.DataFrame(x).replace(token_dict).fillna(-1)+1
        y = pd.DataFrame(y).replace(token_dict).fillna(-1)+1
        x.insert(0, 'start', 1)
        x.insert(x.shape[1], 'end', 2)
        y.insert(0, 'start', 1)
        y.insert(y.shape[1], 'end', 2)
        batch_xs = torch.LongTensor(x.values)
        batch_ys = torch.LongTensor(y.values)
        return batch_xs, batch_ys
    
    
batch_size = 4
datasets = [MyDataset('electron_line_pairs.csv', batch_size), MyDataset('lantern_line_pairs.csv', batch_size)]
z = DataLoader(ConcatDataset(datasets), batch_size=batch_size, shuffle=True, collate_fn=batch_collate_fn)
# for i, (x, y) in enumerate(z):
# #     continue
# # #     print(i)
#     print("the x: %s and y: %s" % (x, y))

FileNotFoundError: [Errno 2] File b'electron_line_pairs.csv' does not exist: b'electron_line_pairs.csv'

In [548]:

count = 0
for i, (x, y) in enumerate(z):
    h = x
    aa = y
    break

In [549]:
x

(['BOS',
  'unknown',
  '_',
  'files',
  '=',
  'self',
  '.',
  'Get',
  'Unknown',
  'Files',
  '(',
  ')',
  'EOS'],
 ['BOS', 'if', 'not', 'files', ':', 'EOS'],
 ['BOS', 'def', 'effective', '_', 'revpair', '(', 'repo', ')', ':', 'EOS'],
 ['BOS', 'promptadd', '(', 'ui', ',', 'repo', ',', 'f', ')', 'EOS'])

In [540]:
x.insert(len(x.columns), 'dprde', -1)

In [543]:
x.shape[1]

14

In [424]:
(pd.DataFrame(x).replace(tokens_dict)+1).fillna(0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,24,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,25,25,12,26,26,26,0,0,0,0,0,0,0,0,0,0,0
2,34,10,11,25,12,36,0,0,0,0,0,0,0,0,0,0,0
3,13,36,15,16,25,17,36,36,25,20,36,24,22,23,24,36,36


In [419]:
tokens_dict

{'import': 23,
 'mimetypes': 1,
 'self': 24,
 '.': 24,
 'reviewer': 11,
 '=': 25,
 '[': 25,
 ']': 25,
 'if': 33,
 'not': 9,
 'cl': 10,
 ':': 35,
 'for': 12,
 'line': 35,
 'in': 14,
 'w': 15,
 'output': 16,
 '(': 35,
 ')': 35,
 'split': 19,
 "'": 23,
 '\\': 21,
 'n': 22,
 'optparse': 23,
 'cc': 24,
 'return': 25,
 'pending': 26,
 'ui': 27,
 ',': 32,
 'repo': 29,
 '*': 32,
 'pats': 31,
 'opts': 32,
 'is': 33,
 'Noise': 34}

In [19]:
# from DataClass.data_utils import read_data, tokenize_fine_grained, get_urls_from_csv

tokenize_fine_grained(x[0])

['import', 'os']

In [424]:
# MyDataset('electron_line_pairs.csv', 1)

In [425]:
# z = DataLoader(ConcatDataset(datasets), batch_size=1, shuffle=False)
for i, (x, y) in enumerate(z):
#     continue
# #     print(i)
    print("the x: %s and y: %s" % (x, y))

the x: tensor([nan], dtype=torch.float64) and y: ('import os',)


In [338]:
datasets = [MyDataset('electron_line_pairs.csv', 1, 2), MyDataset('lantern_line_pairs.csv', 1, 

SyntaxError: unexpected EOF while parsing (<ipython-input-338-96a581c0152e>, line 1)

In [333]:
all_lines.shape[0]

10

In [156]:
# # dataset, dataset_edit = generate_datasets(opt)
# sources, num_file = read_data(opt.path)
# dataset = construct_dataset(sources)#generate_datasets(opt)

In [311]:
x

,0
0,
1,import os
2,import sys
3,def main(args):
4,for dirname in args:
5,try:
6,os.makedirs(dirname)
7,except OSError as e:
8,if e.errno == os.errno.EEXIST:


In [312]:
y

,line
0,import os
1,import sys
2,def main(args):
3,for dirname in args:
4,try:
5,os.makedirs(dirname)
6,except OSError as e:
7,if e.errno == os.errno.EEXIST:
9,main(sys.argv[1:])


In [274]:
y[y['line'].apply(lambda x: len(str(x).replace(' ', '')) > 0)]

,line
0,import os
1,import sys
2,def main(args):
3,for dirname in args:
4,try:
5,os.makedirs(dirname)
6,except OSError as e:
7,if e.errno == os.errno.EEXIST:
9,main(sys.argv[1:])


In [271]:
y['line'][8]

'        '

In [267]:
# y = y[y.apply(lambda x: len(x) > 0)]
y[(len(str(y['liney'])) > 0)]

# df[
# df['names'].apply(lambda x: len(x)>1) &
# df['cars'].apply(lambda x: "i" in x) &
# df['age'].apply(lambda x: int(x)<2)
#   ]

KeyError: True

In [159]:
def extract_examples_line_by_line(source,
                                  n=opt.n_leftmost_tokens,
                                  min_len=opt.min_len):  # If line, no need to filter based on length
    examples = []

    lines = get_lines_from_source(source,
                                  remove_comments_from_source=True,
                                  remove_empty_lines_from_source=True)
    if not lines:
        return []
    
    return lines

In [219]:
hey = extract_examples_line_by_line(dataset)

y = pd.DataFrame(hey)
x = pd.concat([pd.DataFrame([""]), y[0][:-1]]).reset_index(drop=True)
pair = pd.concat([x, y], axis=1)

In [153]:
opt = {"debug": True, "save": True, "overwrite": True, "verbose": True, "unit": "line", "min_len": 3, "distance_metric_x": "nc",
          "distance_threshold_x": 0.3, "distance_metric_y": "c", "distance_threshold_y": 2, "n_leftmost_tokens": 1, "max_num_candidates": 1,
      "check_exact_match_suffix": True}
from collections import namedtuple
MyStruct = namedtuple('MyStruct', 'path debug save overwrite verbose unit min_len distance_metric_x distance_threshold_x distance_metric_y distance_threshold_y n_leftmost_tokens max_num_candidates check_exact_match_suffix')

opt = MyStruct(path = path, debug = True, save = True, overwrite = True, verbose = True, unit = "line", min_len = 10, distance_metric_x = "nc",
          distance_threshold_x = 0.3, distance_metric_y = "c", distance_threshold_y = 2, n_leftmost_tokens = 1, max_num_candidates = 1, 
               check_exact_match_suffix = True)

In [104]:
hey[1315]

IndexError: list index out of range

In [150]:
###############################################################################
# Read data
###############################################################################

def preprocess_source(source):
    """
    Remove # in strings for comment_ptr.

    TODO. Later on, fix regex instead.
    """
    for string in re.findall(r'".*?#.*?"|\'.*?#.*?\'', source):
        if '#' in string:
            modified = string.replace('#', '')
            source = source.replace(string, modified)
    return source


def read_data(path):
    if not os.path.exists(path):
        print("[!] Data does not exist")
    elif os.path.isfile(path):
        return read_file(path)
    else:
        return read_dir(path)


def read_file(path):
    num_file = 1
    sources = []
    try:
        with open(path, 'r') as f:
            source.append(preprocess_source(f.read()))
    except Exception as e:
        print(e)
    return sources, num_file


def read_dir(path):
    num_file = 0
    sources = []
    for filename in glob.iglob(os.path.join(path, "**/*.py"), recursive=True):
#         print(filename)
        num_file += 1
        with open(filename, "r") as f:
            sources.append(preprocess_source(f.read()))
    return sources, num_file


def read_from_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data


def save_as_pickle(path, data):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

In [155]:
# def construct_dataset(sources):
#     """
#     From a project, extract all (x, y) pairs
#     """
#     dataset = []
#     num_lines = 0
#     for source in sources:

#         num_lines += len(get_lines_from_source(source,
#                                                remove_comments_from_source=True,
#                                                remove_empty_lines_from_source=True))
#         return source
#         examples = extract_examples_line_by_line(source)
# #         print(examples)
#         dataset.extend(examples)

#     print(f"\n[construct_dataset] Number of lines in the project: {num_lines}")
#     print(f"[construct_dataset] Number of examples: {len(dataset)} ({len(set(dataset))} unique examples)")
#     return dataset

In [88]:
def generate_datasets(opt):
    """
    Check if processed datasets exist.

    If so, read. Otherwise, generate and save to the path.
    """
    print(f"\n[generate_datasets] Processing {opt.path}")
    options = to_string_opt(opt)
    path_dataset = os.path.join(opt.path, f"dataset_{options}.p")
    path_dataset_edit = os.path.join(opt.path, f"dataset_edit_{options}.p")

    if os.path.exists(path_dataset) and os.path.exists(path_dataset_edit) and not opt.overwrite:
        print("[generate_datasets] Read from pickled files")
        dataset = read_from_pickle(path_dataset)
        dataset_edit = read_from_pickle(path_dataset_edit)
    else:
        sources, num_file = read_data(opt.path)

        dataset = construct_dataset(sources)  # D_proj = {(x, y)}
        if opt.save:
            save_as_pickle(path_dataset, dataset)
            save_as_pickle(path_dataset_edit, dataset_edit)

    print(f"\n[generate_datasets] Number of examples: {len(dataset)} ({len(set(dataset))} unique examples)")

    return dataset

In [12]:
###############################################################################
# Utils
###############################################################################

def rand_select(list, k):
    print(f"Randomly selected {k} examples from {len(list)} examples:")
    return random.choices(list, k=k)


def strip_empty_lines(s):
    """
    Remove empty lines at first and last.
    """
    lines = s.splitlines()
    while lines and not lines[0].strip():
        lines.pop(0)
    while lines and not lines[-1].strip():
        lines.pop()
    return '\n'.join(lines)


def split_newlines(s):
    """
    Split based on new lines (\n) outside of quotes.

    Note that this coalesces several newlines into one,
    as blank lines are ignored. To avoid that, give a null case:

    (?:"[^"]*"|.)+|(?!\Z)
    """
    return re.findall(newline_ptr, s)


def remove_null(l):
    return list(filter(None, l))


def remove_comments(source):
    return re.sub(r"\n\n+", "\n\n", re.sub(comment_ptr, "", source))


def remove_redundant_indentation(code):
    lines = split_newlines(code)
    redundant_indentation = min([len(line) - len(line.lstrip())
                                 for line in lines
                                 if len(line.strip()) > 0])
    lines = [line[redundant_indentation:] for line in lines]
    return lines


def to_string_opt(opt):
    """
    Generate a string for filename that contains current options

    E.g. u_line__d_metric_n__d_thre_0.5__n_1__max_can_5
    """
    s = []
    s.append(f'u_{opt.unit}')
    s.append(f'd_metric_x_{opt.distance_metric_x}')
    s.append(f'd_thre_x_{opt.distance_threshold_x}')
    s.append(f'd_metric_y_{opt.distance_metric_y}')
    s.append(f'd_thre_y_{opt.distance_threshold_y}')
    s.append(f'n_{opt.n_leftmost_tokens}')
    s.append(f'max_can_{opt.max_num_candidates}')
    return '__'.join(s)


def get_lines_from_source(source,
                          remove_comments_from_source,
                          remove_empty_lines_from_source):
    """
    Remove comments and empty lines from source.
    Return a list of lines
    """
    if remove_comments_from_source:
        source = remove_comments(source)

    lines = split_newlines(source)

    if remove_empty_lines_from_source:
        lines = [line for line in lines if len(line.strip()) > 0]
    return lines

In [13]:
###############################################################################
# Tokenize
###############################################################################

def tokenize(s,
             split_camelcase,
             split_number_from_alpha,
             keep_literal,
             keep_whitespace,
             verbose=False):

    numbers, strings, delimiters = [], [], []

    if keep_literal:
        numbers = remove_null(set(re.findall(number_ptr, s)))
        strings = remove_null(set(re.findall(string_ptr, s)))

        literals = [f"(?<=[^a-zA-Z0-9]){re.escape(l)}|^{re.escape(l)}" for l in numbers]  # Add negative look ahead to exclude cases like fc1
        literals.extend([re.escape(l) for l in strings])

        delimiters = sorted(literals, key=len, reverse=True)

    # Basic tokenization based on non alphanumeric tokens
    delimiters.append("[^a-zA-Z0-9]")  # Be careful of the order
    delimiters = remove_null(delimiters)

    tmp_code_ptr = "({})".format("|".join(delimiters))
    tokens = remove_null(re.split(tmp_code_ptr, s))
    if verbose:
        print('[tokenize] Basic:', tokens)

    if split_camelcase:
        before = tokens
        tokens = []
        for token in before:
            if not token:
                continue
            elif token in numbers or token in strings:
                tokens.append(token)
            else:
                tokens.extend(re.split(camelcase_ptr, token))
        tokens = remove_null(tokens)
        if verbose:
            print('[tokenize] Split camel cases:', tokens)

    if split_number_from_alpha:
        before = tokens
        tokens = []
        for token in before:
            if not token:
                continue
            elif token in numbers or token in strings:
                tokens.append(token)
            else:
                tokens.extend(re.split(number_with_alpha_ptr, token))
        tokens = remove_null(tokens)
        if verbose:
            print('[tokenize] Split numbers from alpha:', tokens)

    if not keep_whitespace:
        tokens = [token for token in tokens if len(token.strip()) > 0]
        if verbose:
            print('[tokenize] Remove whitespace:', tokens)

    return tokens


def tokenize_fine_grained(s, keep_whitespace=False):
    """
    Tokenize as much as possible. Used when calculating edit distance.

    E.g., camelCase45 = "hi there" -> camel, Case, 45, ", hi, there, "
    """
    return tokenize(s,
                    split_camelcase=True,
                    split_number_from_alpha=True,
                    keep_literal=False,
                    keep_whitespace=keep_whitespace)


def tokenize_keywords(s):
    """
    Tokenize as much as human-preferable. Used when generating keywords.

    Do not tokenize based on literals.
    By default, whitespace is entirely removed.
    """
    return tokenize(s,
                    split_camelcase=True,
                    split_number_from_alpha=True,
                    keep_literal=True,
                    keep_whitespace=False)


def get_prefix(y,
               n=1):#opt.n_leftmost_tokens):
    return tuple(tokenize_keywords(y)[:n])


def get_suffix(x,
               n=1):#opt.n_leftmost_tokens):
    return tuple(tokenize_keywords(x)[-n:])

In [14]:
###############################################################################
# Print stuff in color
###############################################################################

def print_example(example):
    x, y = example
    print(colored(x, 'red') + colored(y, 'blue'))


def print_examples(examples):
    print("-------------------------------")
    for example in examples:
        print_example(example)
        print("-------------------------------")


def print_candidates(example, candidates):
    print('[x] -------------------------------')
    print(colored(example[0], 'red') + colored(example[1], 'blue'))
    for i, (edit_distance, x, y) in enumerate(candidates, 1):
        print(f"[{i}] {edit_distance:.2f} --------------------------")
        print_example((x, y))


def print_example_edit(example_edit):
    x, y_abs, x_prime, y_prime = example_edit
    print_example((x_prime, y_prime))
    print(colored('-------------------------------', 'white'))
    print_example((x, y_abs))


def print_dataset_edit(dataset_edit):
    for i, example_edit in enumerate(dataset_edit, 1):
        print(f"[{i}] -------------------------------")
        print_example_edit(example_edit)


def plot_histogram(array, title="", xlabel="", ylabel=""):
    n, bins, patches = plt.hist(array, bins=30, facecolor='g', alpha=0.75)
    plt.grid(True)

    if title:
        plt.title(title)
    if xlabel:
        plt.xlabel(xlabel)
    if ylabel:
        plt.ylabel(ylabel)
    plt.show()

In [15]:
###############################################################################
# Extract (x, y) pairs
###############################################################################

def process_example(x, y, n=opt.n_leftmost_tokens):
    """
    Cut and paste the prefix of y at the end of x
    """
    prefix_tokens = get_prefix(y, n=n)
    tmp_prefix_ptr = '\s*' + '\s*'.join([re.escape(token) for token in prefix_tokens])
    prefix_index = re.search(tmp_prefix_ptr, y).end()
    x += '\n' + y[:prefix_index]  # Valid only if each unit is separated by lines
    y = y[prefix_index:]
    return x, y


def reverse_process_example(x, y, n=opt.n_leftmost_tokens):
    """
    Cut and paste the suffix of x at the beginning of y
    """
    suffix_tokens = get_suffix(x, n=n)
    tmp_suffix_ptr = '\s*' + '\s*'.join([re.escape(token) for token in suffix_tokens]) + '$'
    prefix_index = re.search(tmp_suffix_ptr, x).start()
    y = strip_empty_lines(x[prefix_index:] + y)
    x = x[:prefix_index]
    return x, y


def extract_examples_line_by_line(source,
                                  n=opt.n_leftmost_tokens,
                                  min_len=opt.min_len):  # If line, no need to filter based on length
    examples = []
    def add_example(x, y):
        x, y = process_example(x, y, n=n)
        examples.append((x, y))

    lines = get_lines_from_source(source,
                                  remove_comments_from_source=True,
                                  remove_empty_lines_from_source=True)
    if not lines:
        return []

    add_example(x="", y=lines[0])  # First example doesn't have context
    for i in range(1, len(lines) - 1):
        add_example(x=lines[i], y=lines[i + 1])

    # Filter out examples that are too short
    examples = [(x, y) for (x, y) in examples if len(x.strip()) > min_len and len(y.strip()) > min_len]

    # Filter out examples that are import stmts
    examples = [(x, y) for (x, y) in examples if 'import' not in x and 'import' not in y]
    return examples

In [16]:
###############################################################################
# Calculate distance
###############################################################################

def abstract(s):
    return re.sub(literal_ptr, ' ', s)


def has_alpha(tokens):
    return any([token.isalpha() for token in tokens])


def iterative_levenshtein(s, t, costs=(1, 1, 1)):
    """
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t

        costs: a tuple or a list with three integers (d, i, s)
               where d defines the costs for a deletion
                     i defines the costs for an insertion and
                     s defines the costs for a substitution
    """
    rows = len(s)+1
    cols = len(t)+1
    deletes, inserts, substitutes = costs

    dist = [[0 for x in range(cols)] for x in range(rows)]
    # source prefixes can be transformed into empty strings
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = row * deletes
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = col * inserts

    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = substitutes
            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + cost) # substitution
    return dist


def backtrack_levenshtein(tokens1, tokens2, dist, verbose=False):
    """
    Edit tokens2 to tokens1
    """
    i = len(tokens1)
    j = len(tokens2)

    replaced_pairs = []  # list of (token2, token1) pairs
    replaced_indices = [] # list of (index2, index1) pairs

#     if verbose:
#         for row in dist:
#             print(row)
#         print(tokens1)
#         print(tokens2)

    while i > 0 or j > 0:
        if j <= 0:
            if verbose:
                print("Insert", tokens1[i - 1])
            i -= 1
            continue
        if i <= 0:
            if verbose:
                print("Delete", tokens2[j - 1])
            j -= 1
            continue

        if tokens1[i - 1] == tokens2[j - 1]:
            if verbose:
                print(f"Same {tokens1[i - 1]} (i={i - 1}, j={j - 1})")
            i -= 1
            j -= 1
        elif j > 0 and dist[i][j] == dist[i][j - 1] + 1:
            if verbose:
                print("Delete", tokens2[j - 1])
            j -= 1
        elif i > 0 and j > 0 and dist[i][j] == dist[i - 1][j - 1] + 1:
            if verbose:
                print(f"Replace {tokens2[j - 1]} with {tokens1[i - 1]}")
            replaced_pairs.append((tokens2[j - 1], tokens1[i - 1]))
            replaced_indices.append((j - 1, i - 1))
            i -= 1
            j -= 1
        elif i > 0 and dist[i][j] == dist[i - 1][j] + 1:
            if verbose:
                print("Insert", tokens1[i - 1])
            i -= 1
        else:
            if verbose:
                print(f"Error: i={i}, j={j}")
            break


    return replaced_pairs, replaced_indices


def collapse_edit_distance(tokens1, tokens2, verbose=False):
    dist = iterative_levenshtein(tokens1, tokens2)
    replaced_pairs, replaced_indices = backtrack_levenshtein(tokens1, tokens2, dist, verbose=verbose)
    if verbose:
        print(replaced_pairs)

    edit_distance = dist[-1][-1]
    collapse = len(replaced_pairs) - len(set(replaced_pairs))  # Do not count same replacement multiple times
    return edit_distance - collapse


def calculate_edit_distance(code_block1,
                            code_block2,
                            ignore_literals,
                            distance_metric,
                            verbose=False):
    if ignore_literals:  # Todo. Just ignore difference in strings if they are substentially different
        block1 = abstract(code_block1)
        block2 = abstract(code_block2)
        if verbose:
            print("[.] Abstracted code blocks:")
            print(block1.strip())
            print(block2.strip())

    else:
        block1 = code_block1
        block2 = code_block2

    # Tokenize
    tokens1 = tokenize_fine_grained(block1, keep_whitespace=False)
    tokens2 = tokenize_fine_grained(block2, keep_whitespace=False)

    if not tokens1 or not tokens2:
        return float('inf')

    if not has_alpha(tokens1) or not has_alpha(tokens2):
        return float('inf')

    if verbose:
        print(tokens1)
        print(tokens2)

    # https://github.com/doukremt/distance
    if distance_metric == "j":
        return distance.jaccard(tokens1, tokens2)
    elif distance_metric == "l":
        return distance.levenshtein(tokens1, tokens2)
    elif distance_metric == "h":
        return distance.hamming(tokens1, tokens2)
    elif distance_metric == "s":
        return distance.sorensen(tokens1, tokens2)
    elif distance_metric == "n":  # Normalized Levenshtein
        return distance.nlevenshtein(tokens1, tokens2)
    elif distance_metric == "c":  # Collapsed Levenshtein edit distance
        return collapse_edit_distance(tokens1, tokens2, verbose=verbose)
    elif distance_metric == "nc":  # Normalized collapsed Levenshtein edit distance
        collapsed = collapse_edit_distance(tokens1, tokens2, verbose=verbose)
        return collapsed / max(len(tokens1), len(tokens2))


def replace_diff_with_placeholders(string1, string2):
    """
    Replace string2-specific tokens with placeholders. Keep original values.

    E.g. string1 = 'self.fc7 = (self.relu7, 4096, 4096, "fc7")'
         string2 = 'self.fc8 = (self.relu8, 4096, 1000, "fc8")'

         return 'self.fc[[8]] = (self.relu[[8]], 4096, [[1000]], "fc[[8]]")'

    Parameters:
        string1 (str): y
        string2 (str): y'; to be abstracted to be consistent with y
    """
    tokens1 = tokenize_fine_grained(string1, keep_whitespace=True)  # Need to keep whitespace
    tokens2 = tokenize_fine_grained(string2, keep_whitespace=True)  # Need to keep whitespace

    dist = iterative_levenshtein(tokens1, tokens2)
    replaced_pairs, replaced_indices = backtrack_levenshtein(tokens1, tokens2, dist)

    replaced = tokens2
    for index2, index1 in replaced_indices:
        replaced[index2] = f'[[{replaced[index2]}]]'
    return ''.join(replaced)


def rank_based_on_distance(opt,
                           example,
                           examples,
                           include_unsatisfying_examples,
                           exclude_same_context,
                           exclude_same_example,
                           n=opt.n_leftmost_tokens,
                           check_exact_match_suffix=opt.check_exact_match_suffix,
                           verbose=False):
    """
    Parameters:
        check_exact_match_suffix (bool): make sure that the suffix of x
            (n_leftmost_tokens tokens) exactly matches
        exclude_unsatisfying_examples (bool): filter out examples whose
            edit distance is larger than distance_threshold
        exclude_same_context (bool): exclude examples whose context exactly matches x
    """
    ranked_examples = []
    x, y = example
    if verbose:
        print("[..] x:", x)

    candidate_examples = examples
    
    # Select examples whose suffix of x exactly matches with that of example
    if check_exact_match_suffix:
        candidate_examples = []
        x_suffix = get_suffix(x, n)
        if verbose:
            print(f"[.] Enforce to have same {n} tokens as suffix: {x_suffix}")
        for x_prime, y_prime in examples:
            x_prime_suffix = get_suffix(x_prime, n)
            if verbose:
                print("[..] x':", x_prime)
                print("[..] Rightmost tokens:", x_prime_suffix, "\n")
            if x_suffix == x_prime_suffix:
                candidate_examples.append((x_prime, y_prime))

    if verbose:
        print(f"[.] Ranking {len(candidate_examples)} examples")
    for x_prime, y_prime in candidate_examples:
        edit_distance_x = calculate_edit_distance(x,
                                                  x_prime,
                                                  distance_metric=opt.distance_metric_x,
                                                  ignore_literals=False)
        edit_distance_y = calculate_edit_distance(y,
                                                  y_prime,
                                                  distance_metric=opt.distance_metric_y,
                                                  ignore_literals=False)

        if include_unsatisfying_examples:  # Include all examples
            ranked_examples.append((edit_distance, x_prime, y_prime))
        elif edit_distance_x <= opt.distance_threshold_x and edit_distance_y <= opt.distance_threshold_y:
            if exclude_same_context and edit_distance_x == 0:
                continue
            if exclude_same_example and edit_distance_x == 0 and edit_distance_y == 0:
                continue
            ranked_examples.append((edit_distance_x, edit_distance_y, x_prime, y_prime))

    return sorted(set(ranked_examples), key=lambda x:(x[1], x[0]))  # Remove duplicates

In [17]:
###############################################################################
# Construct datasets
###############################################################################

def construct_examples_with_suffix(dataset):
    """
    To speed up, construct clusters of examples based on their suffix
    """
    examples_with_suffix = collections.defaultdict(list)
    for example in dataset:
        x = example[0]
        x_suffix = get_suffix(x)
        examples_with_suffix[x_suffix].append(example)
    return examples_with_suffix


def construct_dataset(sources):
    """
    From a project, extract all (x, y) pairs
    """
    dataset = []
    num_lines = 0
    for source in sources:
        num_lines += len(get_lines_from_source(source,
                                               remove_comments_from_source=True,
                                               remove_empty_lines_from_source=True))
        examples = extract_examples_line_by_line(source)
        dataset.extend(examples)

    print(f"\n[construct_dataset] Number of lines in the project: {num_lines}")
    print(f"[construct_dataset] Number of examples: {len(dataset)} ({len(set(dataset))} unique examples)")
    return dataset


def construct_dataset_edit(opt,
                           dataset,
                           verbose=False):
    """
    Given (x, y) pairs, generate (x, y_abs, x', y')
    """
    dataset_edit = []
    num_examples_with_candidates = 0
    examples_with_suffix = construct_examples_with_suffix(dataset)

    print("\n[construct_dataset_edit] Start generating dataset for edit")
    for example in tqdm(dataset):
        x, y = example
        x_suffix = get_suffix(x)
        candidates = rank_based_on_distance(opt,
                                            example,
                                            examples_with_suffix[x_suffix],
                                            check_exact_match_suffix=False,  # No need to check if passing examples_with_suffix
                                            include_unsatisfying_examples=False,  # Difference: filter out
                                            exclude_same_context=False,
                                            exclude_same_example=True,
                                            verbose=verbose)
        if not candidates:
            pass
        else:
            num_examples_with_candidates += 1
            for candidate in candidates[:opt.max_num_candidates]:
                edit_distance_x, edit_distance_y, x_prime, y_prime = candidate
                y_abs = y  # TODO
                dataset_edit.append((x, y_abs, x_prime, y_prime))
    print(f"[construct_dataset_edit] Number of examples covered for edit: {num_examples_with_candidates}/{len(dataset)} ({num_examples_with_candidates/len(dataset)*100:.2f}%)")
    print(f"[construct_dataset_edit] Number of examples for edit: {len(dataset_edit)} ({len(set(dataset_edit))} unique examples for edit)")
    return dataset_edit


def generate_datasets(opt):
    """
    Check if processed datasets exist.

    If so, read. Otherwise, generate and save to the path.
    """
    print(f"\n[generate_datasets] Processing {opt.path}")
    options = to_string_opt(opt)
    path_dataset = os.path.join(opt.path, f"dataset_{options}.p")
    path_dataset_edit = os.path.join(opt.path, f"dataset_edit_{options}.p")

    if os.path.exists(path_dataset) and os.path.exists(path_dataset_edit) and not opt.overwrite:
        print("[generate_datasets] Read from pickled files")
        dataset = read_from_pickle(path_dataset)
        dataset_edit = read_from_pickle(path_dataset_edit)
    else:
        sources, num_file = read_data(opt.path)

        dataset = construct_dataset(sources)  # D_proj = {(x, y)}
        return dataset
        if len(dataset) > 10000:
            print(f"[generate_datasets] Skipping too large project (|dataset| = {len(dataset)})")
            dataset_edit = []
        else:
            dataset_edit = construct_dataset_edit(opt, dataset)  # D_edit = {(x, y, x', y')}

            # Save datasets for future use
            if opt.save:
                save_as_pickle(path_dataset, dataset)
                save_as_pickle(path_dataset_edit, dataset_edit)

    print(f"\n[generate_datasets] Number of examples: {len(dataset)} ({len(set(dataset))} unique examples)")
    print(f"[generate_datasets] Number of examples for edit: {len(dataset_edit)} ({len(set(dataset_edit))} unique examples)\n")

    return dataset, dataset_edit


In [20]:
###############################################################################
# Render html
###############################################################################

ignorable = '(\s*""".*?"""\s*|\s*\'\'\'.*?\'\'\'\s*|\s)*'


def generate_html(sources):
    html = ""
    for source in sources:
        if len(source.strip()) == 0:
            continue
        html += f"#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#\n\n{source}\n\n"
    return html


def collect_code_with_edits(dataset_edit, html):
    """
    Collect all exact code (~ x + y) that have example edits in dataset_edit.
    """
    # TODO Ignore one liner comments starting with #
    code_with_edits = set()
    cnt_not_found, cnt_found = 0, 0
    for x, y, x_prime, y_prime in dataset_edit:  # NOTE y is not abstracted
        x, y = reverse_process_example(x, y)  # Cut and paste the suffix of x to the beginning of y
        code_to_find_ptr = re.escape(x.strip()) + ignorable + re.escape(y.strip())  # Add ignorable
        code_to_find_ptr = re.compile(code_to_find_ptr, re.MULTILINE|re.DOTALL)

        # Find actual code from html
        found = re.search(code_to_find_ptr, html)
        if not found:
            print("[!] code block not found in source code")
            print(x)
            print(y)
            cnt_not_found += 1
        else:
            code = found.group(0).strip()
            code_with_edits.add(code)
            cnt_found += 1
    print("\nMark code with edits in html")
    print("Found:", cnt_found)
    print("Not found:", cnt_not_found)
    return sorted(code_with_edits, key=len, reverse=True)

In [41]:
###############################################################################
# Main
###############################################################################

def main(opt):
    print("[main] Executing main function with options")
    sources, num_files = read_data(opt.path)
    dataset, dataset_edit = generate_datasets(opt)

    html = generate_html(sources)  # For rendering
    code_with_edits = collect_code_with_edits(dataset_edit, html)  # For rendering
    examples_with_suffix = construct_examples_with_suffix(dataset)  # For metadata

    print("\nTotal number of files processed:", num_files)

    print("\nDistance metric (x):", opt.distance_metric_x)
    print("Distance threshold (x):", opt.distance_threshold_x)
    print("Distance metric (y):", opt.distance_metric_y)
    print("Distance threshold (y):", opt.distance_threshold_y)

    # print("\nNumber of leftmost tokens for keywords (n):", opt.n_leftmost_tokens)
    # print("Maximum number of candidates to generate example edits (k):", opt.max_num_candidates)
    # print("Enforce exact match of the suffix of x and x':", opt.check_exact_match_suffix)

    print(f"\nTotal number of examples: {len(dataset)}")
    print(f"Total number of example edits: {len(dataset_edit)} ({len(dataset_edit) / len(dataset) * 100:.2f}%)")
    print(f"Total number of code with edits: {len(code_with_edits)}", "\n")

    return {
        'sources': sources,
        'num_files': num_files,
        'dataset': dataset,
        'dataset_edit': dataset_edit,
        'html': html,
        'code_with_edits': code_with_edits,
        'examples_with_suffix': examples_with_suffix
    }


# Execute main function with default options
results = main(opt)



[main] Executing main function with options


AttributeError: 'dict' object has no attribute 'path'

In [42]:
###############################################################################
# Flask
###############################################################################

@app.route("/apply_options", methods=["POST"])
def apply_options():
    options = request.get_json(force=True)
    print("\n[apply_options] Applying requested options:", options)

    # Update opt values
    global opt
    opt.distance_metric_x = options['distance_metric_x']
    opt.distance_threshold_x = float(options['distance_threshold_x'])
    opt.distance_metric_y = options['distance_metric_y']
    opt.distance_threshold_y = float(options['distance_threshold_y'])

    # Update processed data for new options
    global results
    results = main(opt)
    print('HERERERERE')
    print(results)
    return render_template("index.html",
                           html=results['html'],
                           dataset_size=len(results['dataset']),
                           dataset_edit_size=len(results['dataset_edit']),
                           coverage=f"{len(results['dataset_edit']) / len(results['dataset']) * 100:.2f}",
                           distance_metric_x=opt.distance_metric_x,
                           distance_threshold_x=opt.distance_threshold_x,
                           distance_metric_y=opt.distance_metric_y,
                           distance_threshold_y=opt.distance_threshold_y)



@app.route("/get_code_with_edits", methods=["POST"])
def get_code_with_edits():
    return jsonify(results['code_with_edits'])


@app.route("/get_metadata", methods=["POST"])
def get_metadata():
    example = request.get_json(force=True)
    print(example)
    # Process example

    x = example['x'].replace("&lt;", "<").replace("&gt;", ">").replace("&amp;", "&")
    y = example['y'].replace("&lt;", "<").replace("&gt;", ">").replace("&amp;", "&")
    # print("Pre POrcexx examples ??????")
    # print(x)
    # print(y)
    x, y = process_example(x, y)
    # print("POrcexx examples ??????")
    # print(x)
    # print(y)

    # Get metadata
    x_suffix = get_suffix(x)
    print("about to process example edits")
    example_edits = rank_based_on_distance(opt,
                                           (x, y),
                                           results['examples_with_suffix'][x_suffix],
                                           check_exact_match_suffix=False,  # No need to check if passing examples_with_suffix
                                           include_unsatisfying_examples=False,
                                           exclude_same_context=False,
                                           exclude_same_example=True)
    print(example_edits)
    print("in between example edits")
    example_edits = [{
                        'edit_distance_x': f'{edit_distance_x:.2f}',
                        'edit_distance_y': f'{edit_distance_y:.2f}',
                        'x': x,
                        'y': y,
                    }
                    for (edit_distance_x, edit_distance_y, x, y) in example_edits]

    data = {
        'x': x,
        'y': y,
        'example_edits': example_edits,
    }
    return jsonify(data)


@app.route("/")
def intro():
    return render_template("index.html",
                           html=results['html'],
                           dataset_size=len(results['dataset']),
                           dataset_edit_size=len(results['dataset_edit']),
                           coverage=f"{len(results['dataset_edit']) / len(results['dataset']) * 100:.2f}",
                           distance_metric_x=opt.distance_metric_x,
                           distance_threshold_x=opt.distance_threshold_x,
                           distance_metric_y=opt.distance_metric_y,
                           distance_threshold_y=opt.distance_threshold_y,
                           min_len=opt.min_len)


if __name__ == "__main__":
    app.run(host="0.0.0.0",  # Public
            debug=opt.debug)


NameError: name 'app' is not defined